# SSD-MobileNet Model For american sign language letters object detection

### install Tensorflow 1.15

In [ ]:
%tensorflow_version 1.x
!pip3 uninstall tensorflow
!pip3 install --user tensorflow-gpu==1.15

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/"

# Clone TensorFlow Model

In [ ]:
!git clone https://github.com/tensorflow/models.git

# Download SSD-MobileNet v2(pre-trained quantized model)

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz

--2020-06-05 10:42:08--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.119.128, 2a00:1450:4013:c00::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.119.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144806142 (138M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz’

ssd_mobilenet_v2_qu 100%[===================>] 138.10M   100MB/s    in 1.4s    

2020-06-05 10:42:09 (100 MB/s) - ‘ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz’ saved [144806142/144806142]



In [ ]:
!tar -xf ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz

In [ ]:
%cd "/content/drive/My Drive/Colab Notebooks/models/research"

In [ ]:
!mv  ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03 ./object_detection/training/ 

# set the PYTHONPATH environment variable

In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Colab Notebooks/models/research:/content/drive/My Drive/Colab Notebooks/models/research/slim/'

!echo $PYTHONPATH

# Compile the Protobuf files
To configure model and training parameters 

In [ ]:
!protoc --python_out=. ./object_detection/protos/anchor_generator.proto ./object_detection/protos/argmax_matcher.proto ./object_detection/protos/bipartite_matcher.proto ./object_detection/protos/box_coder.proto ./object_detection/protos/box_predictor.proto ./object_detection/protos/eval.proto ./object_detection/protos/faster_rcnn.proto ./object_detection/protos/faster_rcnn_box_coder.proto ./object_detection/protos/grid_anchor_generator.proto ./object_detection/protos/hyperparams.proto ./object_detection/protos/image_resizer.proto ./object_detection/protos/input_reader.proto ./object_detection/protos/losses.proto ./object_detection/protos/matcher.proto ./object_detection/protos/mean_stddev_box_coder.proto ./object_detection/protos/model.proto ./object_detection/protos/optimizer.proto ./object_detection/protos/pipeline.proto ./object_detection/protos/post_processing.proto ./object_detection/protos/preprocessor.proto ./object_detection/protos/region_similarity_calculator.proto ./object_detection/protos/square_box_coder.proto ./object_detection/protos/ssd.proto ./object_detection/protos/ssd_anchor_generator.proto ./object_detection/protos/string_int_label_map.proto ./object_detection/protos/train.proto ./object_detection/protos/keypoint_box_coder.proto ./object_detection/protos/multiscale_anchor_generator.proto ./object_detection/protos/graph_rewriter.proto ./object_detection/protos/calibration.proto ./object_detection/protos/flexible_grid_anchor_generator.proto


# Fine-tune the Model

In [ ]:
!python3 ./object_detection/model_main.py --logtostderr --pipeline_config_path=./object_detection/training/ssd_mobilenet_v2_quantized_300x300_coco.config --model_dir=./object_detection/training/ --num_train_steps=200000 --NUM_EVAL_STEPS=10 --alsologtostderr

# Tensorboard for visualization

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./object_detection/training

# Export frozen inference graph for TensorFlow Lite

In [ ]:
!python3 object_detection/export_tflite_ssd_graph.py --pipeline_config_path=./object_detection/training/ssd_mobilenet_v2_quantized_300x300_coco.config  --trained_checkpoint_prefix=object_detection/training/model.ckpt-1247 --output_directory=object_detection/tflite_raspb/ --add_postprocessing_op=true


# TFlite Convert

In [ ]:
!tflite_convert \
  --output_file=./object_detection/TFLite_model/detect_final.tflite \
  --input_shapes=1,300,300,3 \
  --graph_def_file=./object_detection/tflite_raspb/tflite_graph.pb \
  --output_format=TFLITE \
  --inference_type=QUANTIZED_UINT8 \
  --mean_values=128 \
  --std_dev_values=128 \
  --change_concat_input_ranges=False \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
  --allow_custom_ops
